In [1]:
#|default_exp toolloop

# Tool loop

In [2]:
#| export
from claudette.core import *
from fastcore.utils import *

from anthropic.types import TextBlock, Message
from anthropic.types.beta.tools import ToolsBetaMessage, tool_use_block

Anthropic provides an [interesting example](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/customer_service_agent.ipynb) of using tools to mock up a hypothetical ordering system. We're going to take it a step further, and show how we can dramatically simplify the process, whilst completing more complex tasks.

We'll start by defining the same mock customer/order data as in Anthropic's example, plus create a entity relationship between customers and orders:

In [3]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

We can now define the same functions from the original example -- but note that we don't need to manually create the large JSON schema, since Claudette handles all that for us automatically from the functions directly. We'll add some extra functionality to update order details when cancelling too.

In [4]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

We're now ready to start our chat.

In [5]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(models[-1], tools=tools)

We'll start with the same request as Anthropic showed:

In [6]:
r = chat('Can you tell me the email address for customer C1?')
print(r.stop_reason)
r.content

tool_use


[ToolUseBlock(id='toolu_01Xg2oYrXGxMZQRebsUBLRKn', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Claude asks us to use a tool. Claudette handles that automatically by just passing back the message:

In [7]:
r = chat(r)
contents(r)

- Retrieving customer C1


'The email address for customer C1 is john@example.com.'

Let's consider a more complex case than in the original example -- what happens if a customer wants to cancel all of their orders?

In [8]:
chat = Chat(models[-1], tools=tools)
r = chat('Please cancel all orders for customer C1 for me.')
print(r.stop_reason)
r.content

tool_use


[TextBlock(text="Okay, let's cancel all orders for customer C1:", type='text'),
 ToolUseBlock(id='toolu_01PoHeL3NLyfugMy1KojJUEF', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

This is the start of a multi-stage tool use process. Doing it manually step by step is inconvenient, so let's write a function to handle this for us:

In [9]:
#| exports
@patch
def toolloop(self:Chat,
             pr, # Prompt to pass to Claude
             max_steps=10, # Maximum number of tool requests to loop through
             temp=0, # Temperature
             maxtok=4096, # Maximum tokens
             stop:Optional[list[str]]=None, # Stop sequences
             trace_func:Optional[callable]=None, # Function to trace tool use steps (e.g `print`)
             cont_func:Optional[callable]=noop, # Function that stops loop if returns False
             **kw):
    "Add prompt `pr` to dialog and get a response from Claude, automatically following up with `tool_use` messages"
    r = self(pr, temp=temp, maxtok=maxtok, stop=stop, **kw)
    for i in range(max_steps):
        if r.stop_reason!='tool_use': break
        if trace_func: trace_func(r)
        r = self(r, temp=temp, maxtok=maxtok, stop=stop, **kw)
        if not (cont_func or noop)(self.h[-2]['content']): break
    if trace_func: trace_func(r)
    return r

We'll start by re-running our previous request - we shouldn't have to manually pass back the `tool_use` message any more:

In [10]:
chat = Chat(models[-1], tools=tools)
r = chat.toolloop('Can you tell me the email address for customer C1?')
r

- Retrieving customer C1


The email address for customer C1 is john@example.com.

<details>

- id: msg_01REww7fpxS1uuEj6m1dCwxA
- content: [{'text': 'The email address for customer C1 is john@example.com.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 732, 'output_tokens': 19}

</details>

Let's see if it can handle the multi-stage process now -- we'll add `show_trace=print` to see each stage of the process:

In [11]:
chat = Chat(models[-1], tools=tools)
r = chat.toolloop('Please cancel all orders for customer C1 for me.', trace_func=print)
r

ToolsBetaMessage(id='msg_017eULH1pMjc39ZuYs1ENPw5', content=[TextBlock(text="Okay, let's cancel all orders for customer C1:", type='text'), ToolUseBlock(id='toolu_016uvMZWi57bxrkpygS817zJ', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 537; Out: 72; Total: 609)
- Retrieving customer C1
ToolsBetaMessage(id='msg_016SmnJxzWy1pDNvS3aozuf4', content=[TextBlock(text="Based on the customer information, it looks like there are 2 orders for customer C1:\n- Order O1 for Widget A\n- Order O2 for Gadget B\n\nLet's cancel both of these orders:", type='text'), ToolUseBlock(id='toolu_01JJx5RFXcJyy6vdpzBV3zvV', input={'order_id': 'O1'}, name='cancel_order', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 745; Out: 107; Total: 852)
- Cancelling order O1
ToolsBetaMes

Both order cancellations were successful. I have now cancelled all orders for customer C1.

<details>

- id: msg_018xiGfzbJdfwm3Vpp5tF41j
- content: [{'text': 'Both order cancellations were successful. I have now cancelled all orders for customer C1.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 933, 'output_tokens': 23}

</details>

OK Claude thinks the orders were cancelled -- let's check one:

In [12]:
chat.toolloop('What is the status of order O2?')

- Retrieving order O2


The status of order O2 is now 'Cancelled' since I successfully cancelled that order earlier.

<details>

- id: msg_014HbcoPNTVjaW1ewhkt7ERJ
- content: [{'text': "The status of order O2 is now 'Cancelled' since I successfully cancelled that order earlier.", 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 1095, 'output_tokens': 26}

</details>

## Code interpreter

Here is an example of using `toolloop` to implement a simple code interpreter with additional tools.

In [20]:
from lmtools.shell import get_shell
from fastcore.meta import delegates
import traceback

In [21]:
@delegates()
class CodeChat(Chat):
    imps = 'os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses'
    def __init__(self, model: Optional[str] = None, ask:bool=True, **kwargs):
        super().__init__(model=model, **kwargs)
        self.ask = ask
        self.tools.append(self.run_cell)
        self.shell = get_shell()
        self.shell.run_cell('import '+self.imps)

We have one additional parameter to creating a `CodeChat` beyond what we pass to `Chat`, which is `ask` -- if that's `True`, we'll prompt the user before running code.

In [22]:
@patch
def run_cell(
    self:CodeChat,
    code:str,   # Code to execute in persistent IPython session
): # Result of expression on last line (if exists); '#DECLINED#' if user declines request to exectute
    "Asks user for permission, and if provided, executes python `code` using persistent IPython session."
    confirm = f'Press Enter to execute, or enter "n" to skip?\n```\n{code}\n```\n'
    if self.ask and input(confirm): return '#DECLINED#'
    try: res = self.shell.run_cell(code)
    except Exception as e: return traceback.format_exc()
    return res.stdout if res.result is None else res.result

We just pass along requests to run code to the shell's implementation. Claude often prints results instead of just using the last expression, so we capture stdout in those cases.

In [12]:
sp = f'''You are a knowledgable assistant. Do not use tools unless needed.
Don't do complex calculations yourself -- use code for them.
The following modules are pre-imported for `run_cell` automatically:

{CodeChat.imps}

Never mention what tools you are using. Note that `run_cell` interpreter state is *persistent* across calls.'''

In [14]:
def get_user(): # Username of current user
    "Get the username of the user running this session"
    return 'Jeremy'

In order to test out multi-stage tool use, we create a mock function that Claude can call to get the current username.

In [23]:
chat = CodeChat(models[1], tools=[get_user], sp=sp) #, ask=False)

Claude gets confused sometimes about how tools work, so we use examples to remind it:

In [24]:
chat.h = [
    'Set `a` to 3414649225', 'a=3414649225',
    'Calculate the square root of `a`', 'math.sqrt(a)',
    '58435.0', 'The square root of `a` is 58435.',
    'What can you do?', 'I am a knowledgable assistant, can run code, and get the username.'
]

Providing a callable to toolloop's `trace_func` let's us print out information during the loop:

In [ ]:
def _show_cts(r):
    for o in r.content:
        if hasattr(o,'text'): print(o.text)
        nm = getattr(o, 'name', None)
        if nm=='run_cell': print(o.input['code'])
        elif nm: print(f'{o.name}({o.input})')

...and toolloop's `cont_func` callable let's us provide a function which, if it returns `False`, stops the loop:

In [13]:
def _cont_decline(c): return c[0]['content']!='#DECLINED#'

Now we can try our code interpreter. We start by asking for a function to be created, which we'll use in the next prompt to test that the interpreter is persistent.

In [17]:
pr = '''Create a 1-line function `checksum` for a string `s`,
that multiplies together the ascii values of each character in `s` using `reduce`.
Minimise chat.'''
chat.toolloop(pr, temp=0.2, cont_func=_cont_decline, trace_func=_show_cts)

from functools import reduce
checksum = lambda s: reduce(lambda x,y: x*y, (ord(c) for c in s))


None

<details>

- id: msg_01QLuiFGwWyDyXue8zoF2MrZ
- content: []
- model: claude-3-sonnet-20240229
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 666, 'output_tokens': 3}

</details>

By asking for a calculation to be done on the username, we force it to use multiple steps:

In [18]:
pr = 'Use it to get the checksum of the username of this session.'
chat.toolloop(pr, temp=0.2, trace_func=_show_cts)

get_user({})
print(checksum('Jeremy'))
The checksum of the username 'Jeremy' is 1134987783204.


The checksum of the username 'Jeremy' is 1134987783204.

<details>

- id: msg_01T7BXuCwmDQs4ZBzaFTPT1K
- content: [{'text': "The checksum of the username 'Jeremy' is 1134987783204.", 'type': 'text'}]
- model: claude-3-sonnet-20240229
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 814, 'output_tokens': 22}

</details>

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()